<a href="https://colab.research.google.com/github/AlexKazmin/SkillFactory-Projects/blob/main/GD_3_%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GD-3. Исследование поведения пользователей


# ЦЕЛЬ
Исследовать поведение пользователей, зарегестрированных в 2018 году, в обновлённом приложении.

# ЗАДАЧИ
1. Понять, как пользователи взаимодействуют с продуктом, и соотнести идеальный путь пользователей с фактическим.
2. Выявить этапы, которые занимают больше всего времени, с тем чтобы поработать над их улучшением.
3. Проанализировать зависимость оплат от прохождения обучения.

# КОНКРЕТНЫЕ ШАГИ 

1. Определить самые распространённые пути прохождения (последовательности) этапов 
в приложении.
2. Посмотреть на среднее время между различными этапами и выделить самые большие временные промежутки.
3. Определить, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:
  * пользователями, которые прошли обучение хотя бы раз;
  * пользователями, которые начали обучение, но не прошли его ни разу;
  * пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

## Описание приложения
Игра Quiz Freeze состоит из следующих этапов:
* Регистрация (Registration) — это обязательный этап. Без регистрации пользователь не может пройти на следующие этапы работы с приложением.
* Старт обучения (Tutorial Start) — опциональный этап. Пользователь после регистрации может перейти к обучению работе с приложением, а может и не перейти. При этом вернуться к обучению можно в любой момент. А ещё можно пройти обучение несколько раз.
* Завершение обучения (Tutorial Finish) может произойти только в случае, если ранее произошло событие «Старт обучения», но при этом пользователь может не завершить обучение.
* Выбор уровня сложности вопросов (Level Choice) — это обязательное событие, которое нужно для того, чтобы перейти к выбору пакетов вопросов. Таким образом, пользователь может не пройти обучение или даже не начинать его, но прежде чем начать отвечать, он обязан выбрать уровень сложности.
* Выбор пакетов вопросов (Pack Choice, другое название Training Choice) — это этап, на котором пользователь выбирает себе бесплатный набор пакетов вопросов, на которые он будет отвечать.
* Покупка платных пакетов вопросов (Purchase) — это факт совершения оплаты за вопросы, которые не доступны в списке бесплатных вопросов. 
 

Интересующие нас данные о действиях пользователей в игре Quiz Freeze хранятся в двух таблицах — event и purchase.

Таблица event хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

Таблица purchase хранит данные об оплатах, которые совершают пользователи.

Получаем данные из файлов, фильтруем и преобразуем типы



In [3]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

'Загружаем файл и оставляем только интересующий нас промежуток времени'

event_df = pd.read_csv('/content/drive/MyDrive/PROJECT-3/7_4_Events (1).csv', sep=',')
cond = (event_df['start_time'] >= '2018-01-01') & (event_df['start_time'] < '2019-01-01') & (event_df['event_type'] == 'registration')
registered = event_df[cond]['user_id'].to_list()
events = event_df[event_df['user_id'].isin(registered)]
events['start_time'] = pd.to_datetime(events['start_time'], format='%Y-%m-%dT%H:%M:%S')

In [4]:
'Пользователи совершившие оплату'

purchase_df = pd.read_csv('/content/drive/MyDrive/PROJECT-3/purchase (1).csv')
purchase_df['event_type'] = 'purchase'
purchase = purchase_df[purchase_df['user_id'].isin(registered)]
purchase['event_datetime'] = pd.to_datetime(purchase['event_datetime'], format='%Y-%m-%dT%H:%M:%S')

Для того чтобы понимать, как пользователи переходят из этапа в этап и на каких из них возникают сложности, опреляем конверсию на каждом из этапов воронки. То есть нужно понять, какой процент пользователей переходит с предыдущего этапа на следующий.

In [5]:
'сколько пользователей совершают событие registration'

events[events['event_type'] == 'registration']['user_id'].nunique()
events['user_id'].nunique()

19926

In [9]:
'всего уникальных пользователей'
events['user_id'].nunique()

19926

Все пользователи совершают событие registration. Это обусловлено тем, что данный этап является обязательным для выполнения и без него пользователь не сможет перейти дальше.



Посмотрим на срез данных по событию tutorial_start.

In [10]:
events[events['event_type'] == 'tutorial_start'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839
51422,80325,tutorial_start,NaN,2018-01-01 19:46:11,31510.0,27834
51434,80337,tutorial_start,NaN,2018-01-02 02:07:07,31511.0,27840
51435,80338,tutorial_start,NaN,2018-01-02 03:03:44,31512.0,27845
51437,80340,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842
51446,80349,tutorial_start,NaN,2018-01-02 07:08:00,31514.0,27845


Посмотрим на количество пользователей, которые совершают событие tutorial_start:

In [13]:
events[events['event_type'] == 'tutorial_start']['user_id'].nunique()

11858

Число пользователей, которые перешли к выполнению обучения, меньше, чем число пользователей, прошедших регистрацию.

Определим процент пользователей, которые перешли к выполнению обучения, и запишем его в переменную percent_tutorial_start_users.

In [14]:
registered_users_count = events[events["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events[events["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


Посмотрим, какое количество пользователей проходит обучение до конца (событие tutorial_finish).



In [15]:
events[events['event_type'] == 'tutorial_finish']['user_id'].nunique()

10250

Рассчитаем процент пользователей, завершивших обучение, среди пользователей, которые начали обучение. Это будет показатель tutorial_completion_rate (коэффициент «завершаемости» обучения).

In [16]:
tutorial_finish_users_count = events[events["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


На данном этапе исследования можно сделать вывод, что в нашем приложении достаточно хороший процент прохождения обучения.



Изучаем level_choice (выбор уровня сложности)

Посмотрим на процент тех, кто доходит до этого этапа.

In [17]:
'количество пользователей, доходивщих до этапа level_choice'

events[events['event_type'] == 'level_choice']['user_id'].nunique()

8342

In [18]:
'рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень сложности, от общего числа зарегистрировавшихся'

level_choice_users_count = events[events["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


Меньше половины пользователей (41.86 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.

Таким образом, для успешной монетизации приложения крайне важно оптимизировать прохождение до этапа выбора сложности.

Изучаем pack_choice

Определим число пользователей, которые совершили событие выбора бесплатного пакета вопросов pack_choice.

In [19]:
events[events['event_type'] == 'pack_choice']['user_id'].nunique()

5737

Итак, 5737 пользователя доходят до этапа выбора бесплатных вопросов. 
Оценим процент таких пользователей от числа пользователей, которые выбрали уровень сложности.

In [20]:
training_choice_users_count = events[events["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


Изучаем покупку платных пакетов

Следующий этап после выбора бесплатного набора вопросов — это возможная покупка платных пакетов. Оценим число пользователей, которые совершили покупку. Для этого нам потребуется датафрейм purchase_df.

In [21]:
purchase['user_id'].nunique()
paying_users_count = purchase["user_id"].nunique()
paying_users_count

1600

Рассчитаем процент пользователей percent_of_paying_users, которые оплатили вопросы, от числа пользователей, которые приобрели бесплатные вопросы:

In [22]:
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


Таким образом, чуть больше четверти пользователей, которые выбирают бесплатные вопросы, в дальнейшем покупают их.



Посмотрим, какой процент составляют покупатели от общего числа зарегистрировавшихся. Для этого создадим переменную purchase_rate.

In [24]:
purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): 8.03%


Итак, 8% зарегистрировавшихся пользователей приобретают платный пакет.

Само по себе это число нам ни о чём не говорит, поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп. И эта информация укажет нам возможные пути оптимизации.

# Анализ уникальных пользовательских путей

Чтобы понимать, какие есть различные последовательности прохождения пользователей по воронке и насколько часто они встречаются, можно воспользоваться таким подходом:

- отсортировать все события по возрастанию во времени
- объединить для каждого пользователя все его события в один список
- подсчитать частоту различных списков.

Объединим датафрейм с событиями с датафреймом по оплатам. Это позволит анализировать все эти события в рамках одной структуры данных.

Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [25]:
purchase['event_type'] = 'purchase'
events = events.rename(columns={"id": "event_id"})
purchase = purchase.rename(columns={"id": "purchase_id"})
total_events_df = pd.concat([events,purchase],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
68555,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
68556,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
68557,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


Для каждого пользователя создадим список, который будет содержать во временной последовательности все события, совершаемые данным пользователем.



In [26]:
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


Объединим данные по столбцу event_type и подсчитаем количество пользователей по каждому пути, и так получим наиболее популярные пути пользователей.

In [27]:
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head()

user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

Как мы видим, среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase.



Посмотрим, какие ещё последовательности содержат в себе оплату.

In [28]:
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Данное наблюдение позволяет выдвинуть гипотезу, что вероятность оплаты зависит от того, проходил ли пользователь обучение.

Далее проверим эту гипотезу.

# Анализ временных промежутков.

Время между этапами может сообщить нам о вовлечённости пользователей в приложение, а также о роли отдельных этапов на развитие этой вовлечённости. Если окажется, что у пользователей, которые проходят обучение, время до выбора вопросов меньше, чем у тех, кто его не проходит, то разработчикам нужно будет придумывать способы, как вовлекать пользователя в прохождение данного этапа.

Определим, какое время проходит между регистрацией и началом обучения.

Первым делом выделим отдельный датафрейм registration_df, который будет содержать только события с event_type = registration. Этот датафрейм будет вспомогательным для определения времени между регистрацией и началом обучения.

In [29]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df['user_id'].value_counts().mean()
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)
registration_df

,user_id,registration_time
0,27832,2018-01-01 03:48:40
1,27833,2018-01-01 04:07:25
2,27834,2018-01-01 08:35:10
3,27835,2018-01-01 11:54:47
4,27836,2018-01-01 13:28:07
...,...,...
66941,47753,2018-12-31 18:58:55
66942,47754,2018-12-31 19:14:08
66947,47755,2018-12-31 21:15:14
66950,47756,2018-12-31 23:17:30


Теперь выделим отдельный датафрейм tutorial_start_df, который будет содержать только события с event_type = tutorial_start (начало обучения).

In [31]:
tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
7,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
8,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
10,80318.0,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836,NaN,NaT,NaN
12,80320.0,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833,NaN,NaT,NaN
15,80323.0,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839,NaN,NaT,NaN
17,80325.0,tutorial_start,NaN,2018-01-01 19:46:11,31510.0,27834,NaN,NaT,NaN
29,80337.0,tutorial_start,NaN,2018-01-02 02:07:07,31511.0,27840,NaN,NaT,NaN
30,80338.0,tutorial_start,NaN,2018-01-02 03:03:44,31512.0,27845,NaN,NaT,NaN
32,80340.0,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842,NaN,NaT,NaN
41,80349.0,tutorial_start,NaN,2018-01-02 07:08:00,31514.0,27845,NaN,NaT,NaN


Определим среднее количество событий tutorial_start на пользователя

In [32]:
tutorial_start_df['user_id'].value_counts().mean()

1.522179119581717

Видим, что за время пользования приложением, пользователи несколько раз начинают обучение, в среднем, 1.5 раза. 

Оставим только первое обучение. Таким образом мы будем искать время между регистрацией и началом первого обучения пользователя.


Создадим датафрейм tutorial_start_df_wo_duplicates, где будет присутствовать только первое обучение. Для этого отсортируем датафрейм по start_time, чтобы сначала шли более ранние события начала обучения, а затем удалим дубликаты по столбцу user_id. Таким образом, для каждого user_id останется только первое событие типа tutorial_start.

In [35]:
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")

tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})
tutorial_start_df_wo_duplicates

,user_id,tutorial_id,tutorial_start_time
7,27836,31505.0,2018-01-01 14:54:40
8,27835,31506.0,2018-01-01 15:00:51
12,27833,31508.0,2018-01-01 17:47:40
15,27839,31509.0,2018-01-01 19:11:36
17,27834,31510.0,2018-01-01 19:46:11
...,...,...,...
66934,47747,49550.0,2018-12-31 10:21:11
66943,47751,49551.0,2018-12-31 19:38:14
66945,47753,49552.0,2018-12-31 20:54:56
66952,47755,49553.0,2019-01-01 00:08:14


Объединяем датафреймы. Объединение позволит для каждого пользователя в рамках одного датафрейма указать и время регистрации, и время начала первого обучения в приложении.

In [36]:
merged_df = registration_df.merge(
    tutorial_start_df_wo_duplicates, on="user_id", how="inner"
)
merged_df["timedelta"] = (
    merged_df["tutorial_start_time"] - merged_df["registration_time"]
)
merged_df.head()
merged_df["timedelta"].mean()
merged_df['timedelta'].describe()

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

Таким образом:

четверть пользователей тратит меньше 1 часа 21 минуту на переход от регистрации к началу обучения; половина всех пользователей тратит между регистрацией и началом обучения менее 3 часов 22 минут.

Такая сегментация пользователей может дать нам важные опорные точки для оптимизации времени. Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время.

Также мы можем отдельно выделить тех пользователей, у которых время больше медианного, чтобы исследовать их детально и понять, что мешает им быстрее дойти до этапа обучения.

Измеряем время обучения

Перейдём к определению времени, которое нужно для прохождения обучения.

Сначала сформируем датафрейм tutorial_finish_df, который содержит события окончания обучения.

In [37]:
tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]
first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()
tutorial_finish_df = tutorial_finish_df[
    tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)
]
tutorial_finish_df['user_id'].value_counts().mean()
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(
    columns={"start_time": "tutorial_finish_time"}
)
merged_df_2 = tutorial_start_df_wo_duplicates.merge(
    tutorial_finish_df, on="user_id", how="inner"
)
merged_df_2["timedelta"] = (
    merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"]
)
merged_df_2.head()
print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


Таким образом, 75 % пользователей проходят обучение менее чем за 5 минут 8 секунд.

Определяем время между registration и level_choice


In [38]:
level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"]
print(level_choice_df["user_id"].value_counts().mean())
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (
    merged_df_3["level_choice_time"] - merged_df_3["registration_time"]
)
print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

1.0
0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


Определяем время между событием выбора уровня сложности тренировки до события выбора набора бесплатных вопросов

In [39]:
pack_choice_df = total_events_df[total_events_df["event_type"] == "pack_choice"]
print(pack_choice_df["user_id"].value_counts().mean())
pack_choice_df = pack_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "pack_choice_time"}
)
merged_df_4 = level_choice_df.merge(pack_choice_df, on="user_id", how="inner")
merged_df_4["timedelta"] = (
    merged_df_4["pack_choice_time"] - merged_df_4["level_choice_time"]
)
print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

1.0
0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


Определяем время между событием выбора бесплатных вопросов и первой оплатой

In [40]:
purchase_df = total_events_df[total_events_df["event_type"] == "purchase"]
print(purchase_df["user_id"].value_counts().mean())
purchase_df = purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)
merged_df_5 = pack_choice_df.merge(purchase_df, on="user_id", how="inner")
merged_df_5["timedelta"] = (
    merged_df_5["purchase_time"] - merged_df_5["pack_choice_time"]
)
print(merged_df_5["timedelta"].mean())
print(merged_df_5["timedelta"].describe())

1.0
3 days 17:46:53.403125
count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object


# Проверка аналитической гипотезы
Ранее мы видвинули гипотезу:

вероятность оплаты зависит от того, проходил ли пользователь обучение.

Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

* пользователи, которые прошли обучение хотя бы раз;
* пользователи, которые начали обучение, но не прошли его ни разу;
* пользователи, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

Ищем пользователей, прошедших обучение хотя бы раз

In [41]:
users_with_finished_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_finish"
]["user_id"].unique()
print(len(users_with_finished_tutorial))

10250


Ищем пользователей, которые начали, но не закончили обучение

In [42]:
users_with_started_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_start"
]["user_id"].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (
    set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
)

print(len(set_users_with_started_tutorial))
print(len(set_users_not_finished_but_started_tutorial))
print(
    len(set_users_with_started_tutorial) - len(set(users_with_finished_tutorial))
    == len(set_users_not_finished_but_started_tutorial)
)

11858
1608
True


Ищем пользователей, сразу выбравших уровень сложности

In [43]:
all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(
    set_users_with_started_tutorial
)
print(len(set_users_not_started_tutorial))
print(
    len(set_all_users) - len(set_users_with_started_tutorial)
    == len(set_users_not_started_tutorial)
)

8068
True


Проверим, что мы правильно разбили пользователей на группы.

Просуммировав длину всех множеств пользователей, мы должны получить исходное количество всех пользователей:

In [44]:
len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial) + len(users_with_finished_tutorial) == len(set_all_users)

True

Считаем пользователей, которые завершили обучение и совершили оплату

In [45]:
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]
print(purchase_df_1['user_id'].nunique())
percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(
    users_with_finished_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)


1447
Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): 14.12%


Посмотрим на оплаты пользователей, начавших, но не закончивших обучение

In [47]:
purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

131
Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): 8.15%


Рассчитаем процент пользователей, ни разу не начинавших обучение, но совершивших оплаты


In [48]:
purchase_df_3 = purchase_df[purchase_df["user_id"].isin(set_users_not_started_tutorial)]
print(purchase_df_3['user_id'].nunique())
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(
    set_users_not_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

22
Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): 0.27%


# Выводы

Как мы видим, процент пользователей, которые завершили обучение и совершили оплату, выше, чем процент пользователей, которые не начали обучение или не закончили его.

Это говорит о том, что факт прохождения обучения влияет на дальнейшую мотивацию использовать приложение. Существенной разницы в среднем чеке при этом не наблюдается, то есть обучение увеличивает вовлечённость, но выбор количества платных пакетов зависит индивидуально от пользователя.